In [1]:
from Utils.loadset import getDataSet
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import tensorflow_probability as tfp
import os
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs:", len(physical_devices)) 

from trainer import Trainer
try:
    from Utils.connection_cfg import *
except Exception as e:
    PSWD = None
    USRN = None
    
from Utils.Data import dataWrapper

Num GPUs: 1
Num GPUs Available: 1


Using TensorFlow backend.


In [10]:
def convLayer(inp,nbrLayer,channel,activation="relu"):
    print(inp)
    assert nbrLayer > 0, "In Function convLayer nbrLayer > 0 ?"
    layer = Conv2D(channel, kernel_size=(3, 3), padding="same",activation="relu") (inp)
    print(layer)
    #layer = Activation(activation)(layer)
    layer = BatchNormalization()(layer)
    
    for i in range(1,nbrLayer):
        layer = Conv2D(channel, kernel_size=(3, 3), padding="same",activation="relu")  (layer)
        #layer = Activation(activation)(layer)
        print(layer)
        layer = BatchNormalization()(layer)
    return layer

def downUnet():
    pass

def FullUnet(input_shape,down_channels=[64,128,256,512,1024],downLayer=2,activation="relu"):

    
    inputs = Input(shape=input_shape)
    #firstLayer = Conv2D(channel, kernel_size=(3, 3), padding="same")
    layer = convLayer(inputs,1,64)
    pool  = MaxPooling2D((2, 2), strides=(2, 2))(layer)
    
    layerArray = [layer]
    
    for i,channel in enumerate(down_channels[1:]):

        layerArray.append(convLayer(pool,downLayer,channel))
        pool  = MaxPooling2D((2, 2), strides=(2, 2))(layerArray[i+1])
        
    
    # Upconv from Latentspace
    upLayer = None
    print(len(layerArray))
    for i in range(len(layerArray)-1,1,-1):
        upLayer = Conv2DTranspose(down_channels[i-1],(3, 3),strides=(2,2),padding="same")(layerArray[i])
        upLayer = Activation(activation)(upLayer)
        upLayer = BatchNormalization() (upLayer)
        upLayer = concatenate([layerArray[i-1], upLayer], axis=3)
        upLayer = convLayer(upLayer,nbrLayer,64)
        
    lastLayer = convLayer(upLayer,down_channels[0],channel)

    output = Conv2D(1, (1, 1))(lastLayer)
    model = Model(inputs=inputs, outputs=output)
    return model


In [4]:
DatasetFolder = "./Data/RAW"
PathToData = os.path.join(DatasetFolder,"MonthPNGData")

dimension = (128,128)
channels = 5
optimizer = Adam( lr = 1e-5 )
tfd = tfp.distributions
def provideData(flatten=False,dimension=dimension,batch_size=10,transform=None,preTransformation=None):

    getDataSet(DatasetFolder,year=[2017],username=USRN,pswd=PSWD)
    train,test = dataWrapper(PathToData,
                            dimension=dimension,
                            channels=channels,
                            batch_size=batch_size,
                            overwritecsv=True,
                            flatten=flatten,
                            onlyUseYears=[2017],
                            transform=transform,
                            preTransformation=preTransformation)
    
    return train,test

def NLL(y_true, y_hat):
    return -y_hat.log_prob(y_true)

train, test = provideData(flatten=False,dimension=dimension,batch_size=1,transform=None,preTransformation=None)

Found Year :  2017 => won't download this year again... please check for consistency
Finished Loading Dataset
 


In [11]:
t_0 = Trainer(FullUnet,
                    "mse",
                    (train,test),
                    batch_size = 1,
                    optimizer=optimizer,
                    dimension = dimension,
                    channels = channels,
                    metrics = ["mse","mae"])

print("len train,val",len(train),len(test))

Tensor("input_4:0", shape=(None, 128, 128, 5), dtype=float32)
Tensor("conv2d_158/Identity:0", shape=(None, 128, 128, 64), dtype=float32)
Tensor("max_pooling2d_15/Identity:0", shape=(None, 64, 64, 64), dtype=float32)
Tensor("conv2d_159/Identity:0", shape=(None, 64, 64, 128), dtype=float32)
Tensor("conv2d_160/Identity:0", shape=(None, 64, 64, 128), dtype=float32)
Tensor("max_pooling2d_16/Identity:0", shape=(None, 32, 32, 128), dtype=float32)
Tensor("conv2d_161/Identity:0", shape=(None, 32, 32, 256), dtype=float32)
Tensor("conv2d_162/Identity:0", shape=(None, 32, 32, 256), dtype=float32)
Tensor("max_pooling2d_17/Identity:0", shape=(None, 16, 16, 256), dtype=float32)
Tensor("conv2d_163/Identity:0", shape=(None, 16, 16, 512), dtype=float32)
Tensor("conv2d_164/Identity:0", shape=(None, 16, 16, 512), dtype=float32)
Tensor("max_pooling2d_18/Identity:0", shape=(None, 8, 8, 512), dtype=float32)
Tensor("conv2d_165/Identity:0", shape=(None, 8, 8, 1024), dtype=float32)
Tensor("conv2d_166/Identity:0

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,1024,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add] name: conv2d_207/kernel/Initializer/random_uniform/